In [15]:
from flightanalysis import Section, get_schedule
import plotly.express as px
from geometry import Points
from geometry import Point
import numpy as np
import pandas as pd
from flightplotting.plots import plotdtw

sec = Section.from_csv("examples/data/P23.csv").append_derived_values().subset(100, 475)
p23 = get_schedule("F3A", "P23")
template = p23.scale_distance(-sec.y.mean()).create_raw_template("right", sec.bvx.mean(), -sec.y.mean(), False )

dist, aligned = Section.align(sec, template, 2)

plotdtw(aligned, p23.manoeuvres)

/home/tom/source/PyFlightCoach/FlightAnalysis/flightanalysis/section/section.py:14: UserWarning:

No Atmosphere Data Available, assuming SL Standard pressure and temperature. 
 This is for temporary back compatibility with old section csvs.

/home/tom/source/PyFlightCoach/FlightAnalysis/flightanalysis/section/section.py:14: UserWarning:

No Atmosphere Data Available, assuming SL Standard pressure and temperature. 
 This is for temporary back compatibility with old section csvs.



In [35]:
from flightanalysis.section.variables import constructs, subset_vars

nw_aligned = aligned.copy(
    wind=Points.full(Point.zeros(), len(aligned.data)),
    bwind=Points.full(Point.zeros(), len(aligned.data)),
)
nw_aligned.data.drop(subset_vars(["flow"] ), axis=1 )
#nw_aligned.data.columns

nw_aligned = nw_aligned.append_flow()

In [121]:
import plotly.graph_objects as go
man = p23.manoeuvres[0]
windoff = man.get_data(nw_aligned)
windon = man.get_data(aligned)


def cltr(sec, name, dash):
    return go.Scatter(
        x=np.degrees(sec.alpha) , 
        y=4.5 * sec.baz / sec.q, mode="markers+lines", 
        line=dict(dash=dash), 
        marker=dict(
            color=sec.z,
            colorbar=dict(title="altitude"),
        ), 
        name=name
    )

fig = go.Figure(
    data=[
        cltr(windon, "with wind estimation", "dash"),
        #cltr(windoff, "without wind estimation", "dot"),
    ],
    layout=dict(height=800, width=800, title="Comparison of Cl vs Alpha estimation with and without wind estimator for {}".format(man.name)),
    
    )
#
fig.update_xaxes(title="alpha, deg")
fig.update_yaxes(title="CN")
fig.show()

In [122]:
man = p23.manoeuvres[0]

subsec = man.get_data(aligned).data

from plotly.subplots import make_subplots

fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(
    go.Line(
        x=subsec.index, 
        y=-4.5 * subsec.baz / (subsec.q * 0.8), 
        name="Cn"
    ), 
    secondary_y=False
)
fig.add_trace(
    go.Line(
        x=subsec.index, 
        y=np.degrees(subsec.alpha), 
        name="alpha"
    ), 
    secondary_y=True
)
fig.update_layout(
    xaxis=dict(title="time, s"),
    yaxis=dict(range=(-0.5, 0.5), title="-Cn"), 
    yaxis2=dict(range=(-10, 10), title="alpha")
)

fig.show()

/home/tom/miniconda3/envs/pyflightcoach/lib/python3.8/site-packages/plotly/graph_objs/_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




In [118]:
from flightplotting.plots import plotsec
plotsec(man.get_data(aligned))